# Game Theory

The package GameTheory.jl provides some basic features to analyze finite normal form games of complete information (some methods for repeated games are also provided but not discussed here). For example the following prisoner's dilemma can be created and analyzed as follows below.

|  |D |C |
|--|--|--|
|D|-5,-5|-1,-10|
|C|-10,-1|-3,-3|

## Creating games

In [2]:
using GameTheory

player1 = Player([-5 -1; -10 -3])
player2 = Player([-5 -1; -10 -3])
g_pd = NormalFormGame((player1, player2))
print(g_pd)

2×2 NormalFormGame{2, Int64}:
 [-5, -5]   [-1, -10]
 [-10, -1]  [-3, -3]

Note that when we generate player i with the Player() function, the first row of the payoffs corresponds to payoffs of player i that he can obtain with his first action. That is, the order of payoffs of player 2 does not correspond to the usual normal form game table but its transpose. If you find this complicated, you will be happy to hear that alternatively you can create a game from the normal form as follows below:

In [5]:
payoff_bimatrix = Array{Int}(undef, 2, 2, 2)
payoff_bimatrix[1, 1, :] = [-5, -5]
payoff_bimatrix[1, 2, :] = [-1, -10]
payoff_bimatrix[2, 1, :] = [-10, -1]
payoff_bimatrix[2, 2, :] = [-3, -3]
g_pd2 = NormalFormGame(payoff_bimatrix)
print(g_pd2)

2×2 NormalFormGame{2, Int64}:
 [-5, -5]   [-1, -10]
 [-10, -1]  [-3, -3]

## Best response

To determine a player's best response we have the function best_response(player,strategyOther).

In [6]:
best_response(g_pd.players[1], 1) #determines player 1's best response to the first action of player 2


1

In [7]:
best_response(g_pd.players[1], [0.5, 0.5]) #determines player 1's best response to the mixed strategy [0.5, 0.5] of player 2

1

## Nash equilibrium

We can check whether a given strategy profile is a Nash equilibrium using the function is_nash(g,a).

In [8]:
is_nash(g_pd,(1,1)) #checks whether each player using his first action is a Nash equilibrium

true

In [9]:
is_nash(g_pd,([0.3, 0.7],[0.4,0.6])) #checks whether player 1 using mixed strategy [0.3, 0.7] and player 2 using mixed strategy [0.4, 0.6] is a NE

false

There are various ways to determine Nash equilibria of a given game. The function pure_nash finds all pure strategy Nash equilibria of a game.

In [7]:
pure_nash(g_pd)

1-element Vector{Tuple{Int64, Int64}}:
 (1, 1)

The output (1,1) means that each player choosing his first action is a Nash equilibrium. Let us check the same in a game with multiple Nash equilibria -- the so-called stag hunt.

|  |S |H |
|--|--|--|
|S|4,4|0,2|
|H|2,0|0,0|

In [2]:
payoff_bimatrix = Array{Int}(undef, 2, 2, 2)
payoff_bimatrix[1, 1, :] = [4, 4]
payoff_bimatrix[1, 2, :] = [0, 2]
payoff_bimatrix[2, 1, :] = [2, 0]
payoff_bimatrix[2, 2, :] = [2, 2]
g_sh = NormalFormGame(payoff_bimatrix)
print(g_sh)
pure_nash(g_sh)

2×2 NormalFormGame{2, Int64}:
 [4, 4]  [0, 2]
 [2, 0]  [2, 2]

2-element Vector{Tuple{Int64, Int64}}:
 (1, 1)
 (2, 2)

We can also determine mixed strategy Nash equilibria using the function hc_solve(g;ntofind=Inf) where ntofind is the number of Equilibria the algorithm should find and Inf means all Nash equilibria in this context. (Note that this takes quite some time even for small games!)

In [9]:
hc_solve(g_sh; ntofind=Inf)

Tracking 5 paths... 100%|███████████████████████████████| Time: 0:00:04
  # paths tracked:                  5
  # non-singular solutions (real):  5 (5)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         5 (5)


3-element Vector{Tuple{Vector{Float64}, Vector{Float64}}}:
 ([0.5, 0.5], [0.5, 0.5])
 ([1.0, 0.0], [1.0, 0.0])
 ([0.0, 1.0], [0.0, 1.0])

That is, we have 3 mixed strategy Nash equilibria in the stag hunt. the first one has both players mixing putting probability 1/2 on each of their actions while the second and third are the pure strategy equilibria we already saw before.

For 2-player games there are also other methods to determine mixed strategy equilibria. Namely, the function lrsnash(g) (which can only be used if payoffs are integers or rationals) and support_enumeration(g).  

In [3]:
lrsnash(g_sh)

3-element Vector{Tuple{Vector{Rational{BigInt}}, Vector{Rational{BigInt}}}}:
 ([1//2, 1//2], [1//2, 1//2])
 ([0, 1], [0, 1])
 ([1, 0], [1, 0])

In [4]:
support_enumeration(g_sh)

3-element Vector{Tuple{Vector{Float64}, Vector{Float64}}}:
 ([1.0, 0.0], [1.0, 0.0])
 ([0.0, 1.0], [0.0, 1.0])
 ([0.5, 0.5], [0.5, 0.5])

## Discrete approximation of continuous action games

GameTheory.jl can only work with finite games. In particular, this implies that the number of possible actions that a player has must be finite. Many games in economics are not like that. For example, in a Cournot game 2 (or more) firms simultaneously choose their quantity $q_i$ which the leads to a market price of $1-q_1-q_2$. We typically assume that firms can choose any quantity in the interval $[0,1]$. If we want to analyze this game with GameTheory.jl we need to dicretize this game, i.e. we will assume that firms can only choose quantities in the set $\{0,0.01,0.02,0.03,\dots,1.0\}$. THe function discreteGame() that I wrote below generates a discrete game from a vector of payoff functions (here the payoff function of firm i is assumed to be $q_i(1-q_1-q_2)$, i.e. Cournot competition with zero costs).

In [26]:
function discreteGame(piFctVec,aVec)
    """
    discreteGame(piFctVec,aVec)

    Arguments:
    - piFctVect is a vector containing the payoff function of player i
      in its ith element. This function assigns to action vector a
      (where a[i] is the action of player i) the payoff of player i at
      a
    - aVec contains the action set of player i in its ith element (as
      a Vector or tuple)

    Returns:
    a normal form game in the NormalFormGame format of the package
    GameTheory.jl
    """
    N = length(aVec) # number of players
    nums_actions = ntuple(j -> length(aVec[j]),N)
    mygame = NormalFormGame(nums_actions)
    for aIndexProfile in CartesianIndices(nums_actions)
        aprofile = [aVec[j][aIndexProfile[j]] for j in 1:N]
        mygame[aIndexProfile] = [piFctVec[i](aprofile) for i in 1:N]
    end
    return mygame
end

piFctVec = [x->x[1]*(1-x[1]-x[2]),x->x[2]*(1-x[1]-x[2])]
aVec = (collect(0:0.01:1.0),collect(0:0.01:1.0))

g_cournot = discreteGame(piFctVec,aVec)
pure_nash(g_cournot)

3-element Vector{Tuple{Int64, Int64}}:
 (34, 34)
 (35, 34)
 (34, 35)

We see that discretization often comes with its own little surprises: while the usual Cournot game has a unique Nash equilibrium, the discretized version has 3 pure strategy Nash equilibria which, however, are very similar/close to one naother as well as to the continuous action Nash equilibrium in which both firms choose $q_i=1/3$.

(Note: searching for mixed strategy Nash equilibria in such large games (i.e. games with so many actions) is not recommended as it takes a very long time.)

## Dominance and iterative elimination of strictly dominated strategies

We can check which actions of a player are strictly dominated (by a mixed strategy!). For example, in the game below

||L|R|
|--|--|--|
|U|3,1|0,0|
|M|1,0|0,1|
|D|0,1|3,0|

player 1's mixed strategy (1/2,0,1/2) strictly dominates $M$.

In [17]:
p1 = Player([3 0;1 0;0 3])
p2 = Player([1 0 1;0 1 0])
g_d = NormalFormGame((p1,p2))
print(g_d)

3×2 NormalFormGame{2, Int64}:
 [3, 1]  [0, 0]
 [1, 0]  [0, 1]
 [0, 1]  [3, 0]

In [11]:
dominated_actions(g_d.players[1]) # or dominated_actions(p1)

1-element Vector{Int64}:
 2

Using the dominated_actions(player) function, we can now define a function that iteratively eliminates strictly dominated actions.

In [25]:
function iesds(game::NormalFormGame)
    g = game
    i = 1 #denotes the player that is currently checked
    while i<=num_players(game)
        dominated = dominated_actions(g.players[i])
        if length(dominated)>=1 #if at least one dominated action was found
            g = delete_action(g,dominated,i) #delete dominated actions of i from the game
            if i==1 #if we eliminated some action(s) of player 1, we move on to player 2
                i=2
            else #if we eliminated some action(s) of player i>1, we restart from player 1
                i=1 
            end
        else
            i=i+1
        end
    end
    return g
end

print(iesds(g_d))

1×1 NormalFormGame{2, Int64}:
 [3, 1]

In the original game g_d, $M$ was strictly dominated by $(1/2,0,1/2)$. After eliminating $M$, $R$ is strictly dominated by $L$ for player 2 in the remaining game. After eliminating $R$, $U$ strictly dominates $D$ and we are left with a "game" in which each player has only one action. 

Let us try IESDS on the Cournot game above:

In [28]:
print(iesds(g_cournot))

2×2 NormalFormGame{2, Float64}:
 [0.1122, 0.1122]  [0.1089, 0.1122]
 [0.1122, 0.1089]  [0.1088, 0.1088]

This was highly successful: From originally 101 actions per player, only 2 survived the procedure. However, you can see that now we have a problem. Which actions did survive? The procedure above only returns the payoffs but not the original names of the surviving actions. Let's fix that.

In [38]:
function iesds_aNames(game::NormalFormGame)
    g = game
    n = num_players(game)
    num_actions_vec = [num_actions(player) for player in game.players] #vector with the number of actions of each player
    surviving_actions = [collect(1:num_actions_vec[j]) for j in 1:n] #vector of vectors that has as its i-th element a vector containing the surviving actions of player i. Of course, we start with all actions for now and delete some later.
    i = 1 #denotes the player that is currently checked
    while i<=n
        dominated = dominated_actions(g.players[i])
        if length(dominated)>=1 #if at least one dominated action was found
            g = delete_action(g,dominated,i) #delete dominated actions of i from the game
            deleteat!(surviving_actions[i],dominated) #delete the dominated actions 
            if i==1 #if we eliminated some action(s) of player 1, we move on to player 2
                i=2
            else #if we eliminated some action(s) of player i>1, we restart from player 1
                i=1 
            end
        else
            i=i+1
        end
    end
    return g,surviving_actions
end

g_cournot_iesds, actions_iesds = iesds_aNames(g_cournot)
println(g_cournot_iesds)
println(actions_iesds)

2×2 NormalFormGame{2, Float64}:
 [0.1122, 0.1122]  [0.1089, 0.1122]
 [0.1122, 0.1089]  [0.1088, 0.1088]
[[34, 35], [34, 35]]


So the actions 34 and 35 survive IESDS in the Cournot game. (These correspond to the quantities 0.33 and 0.34 and are the quantities that are used in Nash equilibrium.)